In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/raw/data.csv', usecols=[
     "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "trip_distance", 
     "RatecodeID", "PULocationID", "DOLocationID", "payment_type", 
     "extra", "total_amount"])
print(df.shape)
# df.head()

(6405008, 10)


In [3]:
#df = df.drop_duplicates()
df = df.dropna()
df = df[df["passenger_count"] > 0]
df = df[df["trip_distance"] > 0]

df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
trip_dur = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
df = df[trip_dur > 0]

df.shape

(6159738, 10)

In [4]:
#时间
df["pickup_ts"] = df["tpep_pickup_datetime"].astype('int64') // 10**9
df["dropoff_ts"] = df["tpep_dropoff_datetime"].astype('int64') // 10**9
df = df.drop(columns=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

# 类别onehot
cat_cols = ['RatecodeID','payment_type','PULocationID','DOLocationID']
# df = pd.get_dummies(df, columns=cat_cols, dummy_na=False)
for col in cat_cols:
    dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False)
    df = pd.concat([df, dummies], axis=1)
    df.drop(columns=[col], inplace=True)

num_cols = ['passenger_count','trip_distance','extra','pickup_ts','dropoff_ts']
for col in num_cols:
    col_min = df[col].min()
    col_max = df[col].max()
    df[col] = (df[col] - col_min) / (col_max - col_min)

df.shape
# df.head()

(6159738, 535)

In [6]:
import os

os.makedirs("../data/processed", exist_ok=True)

# 保存为 parquet
df.to_parquet("../data/processed/data.parquet", engine="pyarrow", index=False)
print("Saved to ../data/processed/data.parquet")



Saved to ../data/processed/data.parquet


In [ ]:
# # 时间转 Unix 秒
# df["pickup_ts"] = df["tpep_pickup_datetime"].astype('int64') // 10**9
# df["dropoff_ts"] = df["tpep_dropoff_datetime"].astype('int64') // 10**9
# df = df.drop(columns=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

# # 类别onehot
# cat_cols = ['RatecodeID','payment_type','PULocationID','DOLocationID']
# df = pd.get_dummies(df, columns=cat_cols, dummy_na=False)

# # 数值归一化
# num_cols = ['passenger_count','trip_distance','extra','pickup_ts','dropoff_ts']
# scaler = MinMaxScaler()
# df[num_cols] = scaler.fit_transform(df[num_cols])

# df.head()

: 